In [1]:
import os

# Your desired directory path
desired_path = "c:/Projects/taxi_demand_predictory/"

# Change the current working directory to the specified path
os.chdir(desired_path)

# Verify by printing the current working directory
print("Current Working Directory: ", os.getcwd())



Current Working Directory:  c:\Projects\taxi_demand_predictory


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import src.config as config

In [2]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/104604
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
# create feature view (if it doesn't exist yet)
# This feature view only uses on feature group, so the query is trivial
try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')


# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view already existed. Skip creation.


In [4]:
ts_data, _ = feature_view.training_data(
    description='Time-series hourly taxi rides',
)

2023-09-17 19:15:51,873 INFO: USE `taxi_demand_ny_featurestore`
2023-09-17 19:15:52,262 INFO: SELECT `fg0`.`pickup_hour` `pickup_hour`, `fg0`.`rides` `rides`, `fg0`.`pickup_location_id` `pickup_location_id`
FROM `taxi_demand_ny_featurestore`.`time_series_hourly_feature_group_1` `fg0`
WHERE `fg0`.`pickup_hour` >= TIMESTAMP '1970-01-01 12:16:40.000' AND `fg0`.`pickup_hour` < TIMESTAMP '2023-09-18 02:15:50.000'


In [5]:
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data

,pickup_hour,rides,pickup_location_id
923167,2023-01-01 00:00:00,0,1
828722,2023-01-01 01:00:00,0,1
1076488,2023-01-01 02:00:00,0,1
287553,2023-01-01 03:00:00,0,1
536772,2023-01-01 04:00:00,0,1
...,...,...,...
244056,2023-09-17 21:00:00,0,265
243565,2023-09-17 22:00:00,1,265
244288,2023-09-17 23:00:00,0,265
244236,2023-09-18 00:00:00,3,265


In [6]:
ts_data[ts_data['pickup_hour'] == '2023-08-01 00:00:00']

,pickup_hour,rides,pickup_location_id


In [27]:
# from src.plot import plot_ts
from typing import Optional, List
import pandas as pd
import plotly.express as px 

def plot_ts(
    ts_data: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):
    """
    Plot time-series data
    """
    ts_data_to_plot = ts_data[ts_data.pickup_location_id.isin(locations)] if locations else ts_data

    fig = px.line(
        ts_data_to_plot,
        x="pickup_hour",
        y="rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()

plot_ts(ts_data, locations=[22])

In [26]:
print(f"Length of ts_data: {len(ts_data)}")
print(f"Content of ts_data: {ts_data[:5]}")  # Print the first 5 records, if it's a list-like object


Length of ts_data: 1330830
Content of ts_data:                 pickup_hour  rides  pickup_location_id
923167  2023-01-01 00:00:00      0                   1
828722  2023-01-01 01:00:00      0                   1
1076488 2023-01-01 02:00:00      0                   1
287553  2023-01-01 03:00:00      0                   1
536772  2023-01-01 04:00:00      0                   1


In [8]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28, # one month
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')

  0%|          | 0/265 [00:00<?, ?it/s]

c:\Projects\taxi_demand_predictory\src\data.py:240: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

c:\Projects\taxi_demand_predictory\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

c:\Projects\taxi_demand_predictory\src\data.py:240: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

c:\Projects\taxi_demand_predictory\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type 

features_and_target.shape=(50350, 675)


In [9]:

features_and_target

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-29 00:00:00,1,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,0.0,...,1.0,2.0,1.0,1.0,0.0,0.0,0.0,2023-01-29 23:00:00,1,0.0
2,2.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.0,2.0,1.0,2.0,0.0,1.0,0.0,2023-01-30 22:00:00,1,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,2.0,0.0,0.0,0.0,2023-01-31 21:00:00,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,1.0,0.0,2.0,1.0,1.0,0.0,2023-02-01 20:00:00,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50345,3.0,4.0,4.0,3.0,3.0,4.0,5.0,3.0,4.0,10.0,...,7.0,5.0,1.0,8.0,11.0,1.0,3.0,2023-09-14 03:00:00,265,2.0
50346,9.0,9.0,7.0,10.0,6.0,3.0,3.0,9.0,8.0,14.0,...,8.0,7.0,2.0,3.0,5.0,3.0,9.0,2023-09-15 02:00:00,265,4.0
50347,0.0,4.0,1.0,4.0,1.0,3.0,4.0,3.0,5.0,8.0,...,6.0,4.0,6.0,6.0,1.0,1.0,5.0,2023-09-16 01:00:00,265,6.0
50348,8.0,1.0,3.0,1.0,3.0,4.0,4.0,3.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-09-17 00:00:00,265,2.0


In [10]:
from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split

# training data -> from January 2023 up until 2 months ago
# test data -> last 2 months
cutoff_date = pd.to_datetime(date.today() - timedelta(days=28*1))

print(f'{cutoff_date=}')

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'   
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

cutoff_date=Timestamp('2023-08-20 00:00:00')
X_train.shape=(42400, 674)
y_train.shape=(42400,)
X_test.shape=(7950, 674)
y_test.shape=(7950,)


,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,average_rides_last_4_weeks


In [11]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [12]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2023-09-17 19:18:28,058] A new study created in memory with name: no-name-1adb3893-df7d-4273-9dc6-eeb84461ee03
c:\Projects\taxi_demand_predictory\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Projects\taxi_demand_predictory\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



[LightGBM] [Warning] feature_fraction is set=0.2432836098506151, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2432836098506151
[LightGBM] [Warning] bagging_fraction is set=0.5814746365504277, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5814746365504277


c:\Projects\taxi_demand_predictory\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Projects\taxi_demand_predictory\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

c:\Projects\taxi_demand_predictory\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[LightGBM] [Warning] feature_fraction is set=0.2432836098506151, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2432836098506151
[LightGBM] [Warning] bagging_fraction is set=0.5814746365504277, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5814746365504277


c:\Projects\taxi_demand_predictory\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Projects\taxi_demand_predictory\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

[I 2023-09-17 19:18:32,126] Trial 0 finished with value: 3.908883758244468 and parameters: {'num_leaves': 43, 'feature_fraction': 0.2432836098506151, 'bagging_fraction': 0.5814746365504277, 'min_child_samples': 60}. Best is trial 0 with value: 3.908883758244468.
c:\Projects\taxi_demand_predictory\src\model.py:16: SettingWithCopyWarning:




...

c:\Projects\taxi_demand_predictory\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Projects\taxi_demand_predictory\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

[I 2023-09-17 19:18:58,142] Trial 3 finished with value: 3.707579777073266 and parameters: {'num_leaves': 89, 'feature_fraction': 0.32058450166559355, 'bagging_fraction': 0.8902625812921674, 'min_child_samples': 14}. Best is trial 3 with value: 3.707579777073266.
c:\Projects\taxi_demand_predictory\src\model.py:16: SettingWithCopyWarning:



[LightGBM] [Warning] feature_fraction is set=0.7265952641724718, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7265952641724718
[LightGBM] [Warning] bagging_fraction is set=0.9812034787578314, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9812034787578314


c:\Projects\taxi_demand_predictory\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Projects\taxi_demand_predictory\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

c:\Projects\taxi_demand_predictory\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[LightGBM] [Warning] feature_fraction is set=0.7265952641724718, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7265952641724718
[LightGBM] [Warning] bagging_fraction is set=0.9812034787578314, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9812034787578314


c:\Projects\taxi_demand_predictory\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Projects\taxi_demand_predictory\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

[I 2023-09-17 19:19:06,108] Trial 4 finished with value: 3.85608135276595 and parameters: {'num_leaves': 43, 'feature_fraction': 0.7265952641724718, 'bagging_fraction': 0.9812034787578314, 'min_child_samples': 63}. Best is trial 3 with value: 3.707579777073266.


In [13]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 89, 'feature_fraction': 0.32058450166559355, 'bagging_fraction': 0.8902625812921674, 'min_child_samples': 14}


In [14]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

c:\Projects\taxi_demand_predictory\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



[LightGBM] [Warning] feature_fraction is set=0.32058450166559355, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.32058450166559355
[LightGBM] [Warning] bagging_fraction is set=0.8902625812921674, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8902625812921674


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x000001D6AB8A6CA0>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.8902625812921674,
                               feature_fraction=0.32058450166559355,
                               min_child_samples=14, num_leaves=89))])

In [15]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=5.3539


c:\Projects\taxi_demand_predictory\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



In [16]:
import joblib
from src.paths import MODELS_DIR

joblib.dump(pipeline, MODELS_DIR / 'model.pkl')

['C:\\Projects\\taxi_demand_predictory\\models\\model.pkl']

In [17]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [18]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor with a bit of hyper-parameter tuning",
    input_example=X_train.sample(),
    model_schema=model_schema
)

model.save(MODELS_DIR / 'model.pkl')

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

c:\Projects\taxi_demand_predictory\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



Model created, explore it at https://c.app.hopsworks.ai:443/p/104604/models/taxi_demand_predictor_next_hour/3


Model(name: 'taxi_demand_predictor_next_hour', version: 3)